In [ ]:
#!/usr/bin/env python3
import sympy as sp
from IPython.display import display, Math

# ---------- symbols ----------
q1, q2, q3 = sp.symbols('q1 q2 q3', real=True)          # q1=waist yaw, q2=hip pitch, q3=knee pitch
L1, L2, L3 = sp.symbols('L1 L2 L3', positive=True, real=True)
hx, hy, hz = sp.symbols('hx hy hz', real=True)          # hip offset (base->hip)

# ---------- homogeneous transforms ----------
def Tx(a):
    return sp.Matrix([[1,0,0,a],
                      [0,1,0,0],
                      [0,0,1,0],
                      [0,0,0,1]])

def Ty(a):
    return sp.Matrix([[1,0,0,0],
                      [0,1,0,a],
                      [0,0,1,0],
                      [0,0,0,1]])

def Tz(a):
    return sp.Matrix([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,1,a],
                      [0,0,0,1]])

def Rx(th):
    c, s = sp.cos(th), sp.sin(th)
    return sp.Matrix([[1,0,0,0],
                      [0,c,-s,0],
                      [0,s, c,0],
                      [0,0,0,1]])

def Ry(th):
    c, s = sp.cos(th), sp.sin(th)
    return sp.Matrix([[ c,0, s,0],
                      [ 0,1, 0,0],
                      [-s,0, c,0],
                      [ 0,0, 0,1]])

def Rz(th):
    c, s = sp.cos(th), sp.sin(th)
    return sp.Matrix([[c,-s,0,0],
                      [s, c,0,0],
                      [0, 0,1,0],
                      [0, 0,0,1]])

# ---------- RR leg chain ----------
# Base -> Hip offset
TBH = Tx(hx) * Ty(hy) * Tz(hz)

T_align = Ry(sp.pi/2)

# Hip yaw (waist) about Z
TH0 = Rz(q1)

# Hip pitch about Y, then link L1 along X
T01 = Ry(q2) * Tx(L1)

# Knee pitch about Y, then link (L2+L3) along X to foot/wheel-center
T12 = Ry(q3) * Tx(L2 + L3)

# Full transform: base -> foot
TBF = sp.simplify(TBH * T_align * TH0 * T01 * T12)
# TBF = sp.simplify(TBH * TH0 * T01 * T12)

# ---------- position ----------
x, y, z = sp.simplify(TBF[0,3]), sp.simplify(TBF[1,3]), sp.simplify(TBF[2,3])
p = sp.Matrix([x, y, z])
display(Math(r"[x,y,z]^T = " + sp.latex(p)))

# ---------- Jacobian (3x3) ----------
J = sp.simplify(p.jacobian([q1, q2, q3]))
display(Math(r"\mathbf{J}=" + sp.latex(J)))

# ---------- orientation (optional) ----------
R = TBF[:3,:3]
roll  = sp.atan2(R[2,1], R[2,2])
pitch = sp.atan2(-R[2,0], sp.sqrt(R[2,1]**2 + R[2,2]**2))
yaw   = sp.atan2(R[1,0], R[0,0])
pose_expr = sp.Matrix([x, y, z, roll, pitch, yaw])
# display(Math(r"[x,y,z,\phi,\theta,\psi]^T=" + sp.latex(pose_expr)))

# ---------- test substitution ----------
subs_test = {
    q1: sp.rad(0),
    q2: sp.rad(-127.10900494580854),
    q3: sp.rad(-94.5406673280583),
    L1: 0.2,
    L2: 0.24,
    L3: 0.0,
    hx: 0.0,
    hy: 0.2,   # RR hip offset (example)
    hz: 0.0
}
p_eval = p.subs(subs_test).evalf()
print("p_eval =", p_eval)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

p_eval = Matrix([[-1.22752878652075e-16], [0.200000000000000], [0.300000000000000]])


In [56]:
#!/usr/bin/env python3
import math
from typing import List, Tuple

def wrap_pi(a: float) -> float:
    """wrap angle to [-pi, pi)"""
    a = (a + math.pi) % (2.0 * math.pi) - math.pi
    return a

def ik_from_fk_chain(
    x: float, y: float, z: float,
    L1: float, L2: float, L3: float,
    hx: float, hy: float, hz: float,
    prefer_q1_near: float = 0.0,
) -> List[Tuple[float, float, float]]:
    """
    IK for the exact FK implied by:
      TBF = TBH * Ry(pi/2) * Rz(q1) * Ry(q2)*Tx(L1) * Ry(q3)*Tx(L2+L3)

    Returns list of (q1,q2,q3) in radians (multiple branches).
    """
    L23 = L2 + L3

    # --- (1) compute s,t,r in yz-plane ---
    s = y - hy
    t = z - hz
    r = math.hypot(s, t)

    # If r ~ 0 => q1 is not observable (any yaw gives same point)
    if r < 1e-12:
        q1_candidates = [(wrap_pi(prefer_q1_near), 0.0)]  # (q1, A)
    else:
        q1a = math.atan2(s, -t)      # q1 with A = +r
        q1b = wrap_pi(q1a + math.pi) # equivalent with A = -r
        q1_candidates = [(wrap_pi(q1a), +r), (wrap_pi(q1b), -r)]

    # --- (2) compute dx from x equation ---
    dx = hx - x  # dx = L1*sin(q2) + L23*sin(q2+q3)

    sols = []
    for q1, A in q1_candidates:
        u = A   # u = L1*cos(q2) + L23*cos(q2+q3)  (signed!)
        v = dx  # v = L1*sin(q2) + L23*sin(q2+q3)

        # reachability
        denom = 2.0 * L1 * L23
        if abs(denom) < 1e-12:
            continue

        D = (u*u + v*v - L1*L1 - L23*L23) / denom
        if D < -1.0 - 1e-9 or D > 1.0 + 1e-9:
            # unreachable for this branch
            continue
        D = max(-1.0, min(1.0, D))

        for elbow in (+1.0, -1.0):
            s3 = elbow * math.sqrt(max(0.0, 1.0 - D*D))
            q3 = math.atan2(s3, D)

            q2 = math.atan2(v, u) - math.atan2(L23*s3, L1 + L23*D)

            sols.append((wrap_pi(q1), wrap_pi(q2), wrap_pi(q3)))

    # Optional: sort by how close q1 is to prefer_q1_near
    # sols.sort(key=lambda q: abs(wrap_pi(q[0] - prefer_q1_near)))
    return sols


# ------------------- quick test (your example) -------------------
if __name__ == "__main__":
    # p_eval from your notebook
    x = 0.0
    y = 0.2
    z = 0.2

    sols = ik_from_fk_chain(
        x,y,z,
        L1=0.2, L2=0.24, L3=0.0,
        hx=0.0, hy=0.2, hz=0.0,
        prefer_q1_near=0.0
    )

    for i,(q1,q2,q3) in enumerate(sols):
        print(i, "deg =", (math.degrees(q1), math.degrees(q2), math.degrees(q3)))


0 deg = (-180.0, -73.73979529168804, 126.86989764584402)
1 deg = (-180.0, 73.73979529168803, -126.86989764584402)
2 deg = (0.0, 106.26020470831197, 126.86989764584402)
3 deg = (0.0, -106.26020470831197, -126.86989764584402)


In [61]:
#!/usr/bin/env python3
import sympy as sp
from IPython.display import display, Math

# ---------- symbols ----------
q1, q2, q3 = sp.symbols('q1 q2 q3', real=True)
L1, L2, L3 = sp.symbols('L1 L2 L3', positive=True, real=True)
hx, hy, hz = sp.symbols('hx hy hz', real=True)
x, y, z    = sp.symbols('x y z', real=True)   # desired foot position in base frame

# ---------- helper ----------
def wrap_pi_sym(a):
    return sp.simplify(sp.Mod(a + sp.pi, 2*sp.pi) - sp.pi)



L23 = sp.simplify(L2 + L3)
s  = sp.simplify(y  - hy)
t  = sp.simplify(z  - hz)
r = sp.simplify(sp.sqrt(s**2 + t**2))   # = |U|


q1a = sp.atan2(s, -t)            # corresponds to U = +r
q1b = sp.simplify(q1a + sp.pi)   # corresponds to U = -r

dx = sp.simplify(hx - x)

# Choose ONLY q1b branch (your requested "q1_candidates = (q1b, -r)")
U = sp.simplify(-r)  # signed

# 2-link planar IK in (u,v) where:
u = U
v = dx

D = sp.simplify((u**2 + v**2 - L1**2 - L23**2) / (2*L1*L23))

# elbow = +1 then -1
s3_pos = sp.simplify(sp.sqrt(1 - D**2))
s3_neg = sp.simplify(-sp.sqrt(1 - D**2))

q3_pos = sp.atan2(s3_pos, D)
q3_neg = sp.atan2(s3_neg, D)

q2_pos = sp.simplify(sp.atan2(v, u) - sp.atan2(L23*s3_pos, L1 + L23*D))
q2_neg = sp.simplify(sp.atan2(v, u) - sp.atan2(L23*s3_neg, L1 + L23*D))

# One-solution policy: take elbow=+1 first (like your for loop) => (q1b, q2_pos, q3_pos)
q1_sol = wrap_pi_sym(sp.simplify(q1b))
q2_sol = wrap_pi_sym(sp.simplify(q2_neg))
q3_sol = wrap_pi_sym(sp.simplify(q3_neg))

display(Math(r"\textbf{IK (one-solution, q1b + elbow=-1)}:"))
display(Math(r"q_1 = " + sp.latex(q1_sol)))
display(Math(r"q_2 = " + sp.latex(q2_sol)))
display(Math(r"q_3 = " + sp.latex(q3_sol)))

# ---------- numeric test (เหมือนของคุณ) ----------
subs_test = {
    L1: 0.2, L2: 0.24, L3: 0.0,
    hx: 0.0, hy: 0.2, hz: 0.0,
    x: 0.0, y: 0.2, z: 0.3
}

q_eval = sp.Matrix([q1_sol, q2_sol, q3_sol]).subs(subs_test).evalf()
print("q_eval (rad) =", q_eval)
print("q_eval (deg) =", [float(sp.deg(q)) for q in q_eval])


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

q_eval (rad) = Matrix([[0], [-2.21847064523812], [-1.65004592190725]])
q_eval (deg) = [0.0, -127.10900494580854, -94.5406673280583]
